In [7]:
import numpy as np
import sys
np.set_printoptions(precision=2, floatmode="fixed")
np.set_printoptions(threshold=sys.maxsize)

In [8]:
def row_swap(mat, permutation):
    """
    Ensures that the element at row 1 column 1 is a nonzero number.
    If the element at row 1 column 1 is zero, it finds the first row 
    with a nonzero element in column 1, and swaps that row with row 1.
    """
    first_col = mat[:, 0]
    nonzero_rows = np.nonzero(first_col)
    # if there are no nonzero rows, end
    if nonzero_rows[0].size == 0:
        return False
    # if the first row is not nonzero, swap
    if nonzero_rows[0][0] != 0:
        mat[[0, nonzero_rows[0][0]], :] = mat[[nonzero_rows[0][0], 0], :]
        permutation[[0, nonzero_rows[0][0]], :] = permutation[[nonzero_rows[0][0], 0], :]
    return True

In [3]:
def build(mat, i, n, m, permutation, lower, upper):
    """
    Builds the ith column of the lower triangular matrix and the
    ith row of the upper triangular matrix. Returns the truncated
    remainder matrix to be used again.
    """
    # If the matrix is empty, can no longer decompose, return matrix
    if not np.any(mat):
        return mat
    
    # Perform row swap. If there are no nonzero rows, no valid column exists
    column_exists = row_swap(mat, permutation)
    
    # Build the row vector for outer product
    row = mat[0, :]
    row_vector = row.reshape(1, row.size)
    
    # Pad the row to make it 1 x m
    row_padding_dim = m - row_vector.shape[1]
    padded_row = np.pad(row_vector, [ (0, 0), (row_padding_dim, 0) ], mode='constant')
    upper[i] = padded_row
    
    # Column vector begins as column from identity matrix
    column_vector = np.transpose([lower[:, i]])
    
    # If column exists, we can build the lower triangular matrix
    if column_exists:
        # Build the column vector for outer product
        column_vector = np.transpose([mat[:, 0] / row_vector[0, 0]])

        # Pad the column to make it n x 1
        col_padding_dim = n - column_vector.shape[0]
        padded_column = np.pad(column_vector, [ (col_padding_dim, 0), (0, 0) ], mode='constant')

        # Modify the lower triangular matrices
        lower[:, i] = np.transpose(padded_column)
    
    # Create the remainder matrix
    remainder_matrix = mat - np.matmul(column_vector, row_vector)
    truncated_remainder = remainder_matrix[1:,1:]

    return truncated_remainder

In [4]:
def decompose(matrix):
    # Create a copy so the original matrix is unchanged
    mat = matrix.copy()
    n, m = mat.shape
    permutation = np.identity(mat.shape[0])
    lower = np.identity(mat.shape[0])
    upper = np.zeros(mat.shape)
    
    # Build the n columns of lower and the n rows of upper
    for i in range(n):
        mat = build(mat, i, n, m, permutation, lower, upper)
        
    return permutation, lower, upper

In [5]:
def testLUandDeterminants(matrix, printing=True):
    # Print the original matrix
    if (printing):
        print("original")
        print(matrix)
    perm, lower, upper = decompose(matrix)
    
    # Print the results
    if (printing):
        print("perm")
        print(perm.astype(int))
        print("lower")
        print(lower)
        print("upper")
        print(upper)
    
    # Validate the results
    correct = True
    mult = perm @ lower @ upper
    if (printing):
        print("check")
        print(mult.astype(int))
    
    # We use allclose instead of equals because of floating point precision
    correct &= np.allclose(mult, matrix)
    if (printing):
        print("Matrix is correct" if correct else ">>> INCORRECT MATRIX <<<")
    
    # Check if determinant is right for square matrices
    if (matrix.shape[0] == matrix.shape[1]):
        perm_diag = np.diagonal(perm)
        lower_diag = np.diagonal(lower)
        upper_diag = np.diagonal(upper)
        
        nswaps = (len(perm_diag) - np.sum(perm_diag)) / 2
        detP = (-1)**nswaps
        detL =  np.prod(lower_diag) 
        detU = np.prod(upper_diag)
        
        our_det = detP * detL * detU
        actual_det = np.linalg.det(matrix)
        # We use isclose instead of equals because of floating point precision
        correct &= np.isclose(our_det, actual_det)
        if (printing):
            print("Determinant is correct" if correct else ">>> INCORRECT DETERMINANT <<<")
    
    if (printing):
        print("-"*20)
    return correct

In [6]:
result = True
# Entire first column is zero
result &= testLUandDeterminants(np.array([[0,-2,-3],[0,-3,-4],[0,7,14]]))
# Entire first column is zero, nonsquare
result &= testLUandDeterminants(np.array([[0,-2],[0,-3],[0,7]]))
# Basic matrix
result &= testLUandDeterminants(np.array([[2,-2,-3],[4,-3,-4],[-6,7,14]]))
# Zero pivot
result &= testLUandDeterminants(np.array([[0,-2,-3],[4,-3,-4],[-6,7,14]]))
# Zero in all diagonals
result &= testLUandDeterminants(np.array([[0,-2,-3],[3,0,-4],[-6,7,0]]))
# Wide matrix
result &= testLUandDeterminants(np.array([[-2,1,3,5,9,4,3,2],[-4,4,1,4,5,1,5,9]]))
# Tall matrix
result &= testLUandDeterminants(np.array([[-2,1], [5,9], [3,4], [5,10],[4,0],[3,2]]))
# Print result
print("All correct" if result else "Incorrect")

original
[[ 0 -2 -3]
 [ 0 -3 -4]
 [ 0  7 14]]
perm
[[1 0 0]
 [0 1 0]
 [0 0 1]]
lower
[[ 1.00  0.00  0.00]
 [ 0.00  1.00  0.00]
 [ 0.00 -2.33  1.00]]
upper
[[ 0.00 -2.00 -3.00]
 [ 0.00 -3.00 -4.00]
 [ 0.00  0.00  4.67]]
check
[[ 0 -2 -3]
 [ 0 -3 -4]
 [ 0  7 14]]
Matrix is correct
Determinant is correct
--------------------
original
[[ 0 -2]
 [ 0 -3]
 [ 0  7]]
perm
[[1 0 0]
 [0 1 0]
 [0 0 1]]
lower
[[ 1.00  0.00  0.00]
 [ 0.00  1.00  0.00]
 [ 0.00 -2.33  1.00]]
upper
[[ 0.00 -2.00]
 [ 0.00 -3.00]
 [ 0.00  0.00]]
check
[[ 0 -2]
 [ 0 -3]
 [ 0  7]]
Matrix is correct
--------------------
original
[[ 2 -2 -3]
 [ 4 -3 -4]
 [-6  7 14]]
perm
[[1 0 0]
 [0 1 0]
 [0 0 1]]
lower
[[ 1.00  0.00  0.00]
 [ 2.00  1.00  0.00]
 [-3.00  1.00  1.00]]
upper
[[ 2.00 -2.00 -3.00]
 [ 0.00  1.00  2.00]
 [ 0.00  0.00  3.00]]
check
[[ 2 -2 -3]
 [ 4 -3 -4]
 [-6  7 14]]
Matrix is correct
Determinant is correct
--------------------
original
[[ 0 -2 -3]
 [ 4 -3 -4]
 [-6  7 14]]
perm
[[0 1 0]
 [1 0 0]
 [0 0 1]]
lower
[[

In [7]:
# Testing the function on square random integer matrices
result = True
for _ in range(100):
    n = np.random.randint(1, 20)
    m = n
    mat = np.random.randint(-100, 100, size=(n, m))
    result &= testLUandDeterminants(mat, printing=False)

print("All correct" if result else "Incorrect")

All correct


In [8]:
# Testing the function on random integer matrices
result = True
for _ in range(100):
    n, m = np.random.randint(1, 20, 2)
    mat = np.random.randint(-100, 100, size=(n, m))
    result &= testLUandDeterminants(mat, printing=False)
print("All correct" if result else "Incorrect")

All correct


In [9]:
"""
Explain how you could use this to compute the determinant of a matrix.

det(PLU) = det(P) * det(L) * det(U)

det(P) is number of row swaps. If the number of row swaps is even, the det is 1. If it's odd, then det is -1.
det(L) is the product of the diagonal entries of the lower-triangular matrix. Based on how we built the LU-decomposition, this will always be 1.
det(U) is the product of the diagonal entries of the upper-triangular matrix.
"""

"\nExplain how you could use this to compute the determinant of a matrix.\n\ndet(PLU) = det(P) * det(L) * det(U)\n\ndet(P) is number of row swaps. If the number of row swaps is even, the det is 1. If it's odd, then det is -1.\ndet(L) is the product of the diagonal entries of the lower-triangular matrix. Based on how we built the LU-decomposition, this will always be 1.\ndet(U) is the product of the diagonal entries of the upper-triangular matrix.\n"